In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import pickle

# Load data
products = pd.read_csv('/mnt/data/Products_ThoiTrangNam_raw.csv')
ratings = pd.read_csv('/mnt/data/Products_ThoiTrangNam_rating_raw.csv', sep='\t')

# Tiền xử lý
products.dropna(subset=['product_name', 'description'], inplace=True)
products['text'] = products['product_name'] + ' ' + products['description'] + ' ' + products['category'] + ' ' + products['sub_category']

# Content-based recommender
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(products['text'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# User-based recommender
user_item_matrix = ratings.pivot_table(index='user_id', columns='product_id', values='rating')
user_item_matrix.fillna(0, inplace=True)
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(user_item_matrix)

# Các hàm gợi ý
def recommend_products_by_product(product_id, top_n=5):
    idx = products[products['product_id'] == product_id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    product_indices = [i[0] for i in sim_scores]
    return products.iloc[product_indices]

def recommend_products_by_user(user_id, top_n=5):
    if user_id not in user_item_matrix.index:
        return pd.DataFrame()
    distances, indices = model_knn.kneighbors(user_item_matrix.loc[user_id].values.reshape(1, -1), n_neighbors=3)
    similar_users = user_item_matrix.index[indices.flatten()].tolist()
    recommended_products = user_item_matrix.loc[similar_users].mean(axis=0)
    recommended_products = recommended_products.sort_values(ascending=False)
    recommended_products = recommended_products.index[:top_n]
    return products[products['product_id'].isin(recommended_products)]

# Streamlit app
st.set_page_config(page_title='Fashion Recommender', page_icon=':shirt:', layout='wide')
st.title('👕 Fashion Recommendation System')

menu = st.sidebar.selectbox('Menu', ['Trang chủ', 'Đề xuất theo sản phẩm', 'Đề xuất theo người dùng'])

if menu == 'Trang chủ':
    st.header('🌟 Top 3 sản phẩm nổi bật')
    top_products = products.sort_values(by='rating', ascending=False).head(3)
    cols = st.columns(3)
    for idx, row in top_products.iterrows():
        with cols[idx % 3]:
            st.image(row['image'], width=200)
            st.subheader(row['product_name'])
            st.caption(f'Giá: {int(row["price"]):,} đ | Rating: {row["rating"]}')
            st.markdown(f'[Xem chi tiết]({row["link"]})')

elif menu == 'Đề xuất theo sản phẩm':
    st.header('🔎 Đề xuất theo sản phẩm')
    selected_product = st.selectbox('Chọn sản phẩm', products['product_name'].values)
    selected_id = products[products['product_name'] == selected_product]['product_id'].values[0]
    recommendations = recommend_products_by_product(selected_id)
    st.subheader('Sản phẩm tương tự:')
    for idx, row in recommendations.iterrows():
        with st.container():
            st.image(row['image'], width=150)
            st.write(f'**{row["product_name"]}**')
            st.caption(f'Giá: {int(row["price"]):,} đ | Rating: {row["rating"]}')
            st.markdown(f'[Mua ngay]({row["link"]})')

elif menu == 'Đề xuất theo người dùng':
    st.header('👤 Đề xuất theo người dùng')
    selected_user = st.selectbox('Chọn người dùng', ratings['user'].unique())
    selected_user_id = ratings[ratings['user'] == selected_user]['user_id'].values[0]
    recommendations = recommend_products_by_user(selected_user_id)
    if recommendations.empty:
        st.warning('Không tìm thấy gợi ý cho người dùng này.')
    else:
        st.subheader('Sản phẩm đề xuất:')
        for idx, row in recommendations.iterrows():
            with st.container():
                st.image(row['image'], width=150)
                st.write(f'**{row["product_name"]}**')
                st.caption(f'Giá: {int(row["price"]):,} đ | Rating: {row["rating"]}')
                st.markdown(f'[Mua ngay]({row["link"]})')
